<a href="https://colab.research.google.com/github/tejaspradhan/Graph-Neural-Networks/blob/main/hiv-project/gnn_explainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer, models
from torch import nn

word_embedding_model = models.Transformer('bert-base-uncased', max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=512, activation_function=nn.Tanh())

bert_model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
!git clone https://github.com/neerajajaja/Face2Text.git

fatal: destination path 'Face2Text' already exists and is not an empty directory.


In [ ]:
from tensorflow.keras.models import load_model
import tensorflow as tf
from keras.applications.xception import Xception
from google.colab import drive
from keras.models import Model
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from keras.applications.xception import preprocess_input
import os

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model=load_model('/content/Face2Text/training_files/model_x_19.h5')

In [ ]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 77)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 2048)]       0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 77, 200)      267200      ['input_3[0][0]']                
                                                                                                  
 dropout_1 (Dropout)            (None, 2048)         0           ['input_2[0][0]']                
                                                                                            

In [ ]:
def greedySearch(photo):
    in_text = 'startseq'
    for i in range(maxv):
        sequence = [wordtoin[w] for w in in_text.split() if w in wordtoin]
        sequence = pad_sequences([sequence], maxlen=maxv)
        yhat = model.predict([photo,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = intoword[yhat]
        in_text += ' ' + word
        if word == 'seqend':
            break
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

In [ ]:
def resize_image(img, target_width=299):
  mywidth = target_width
  wpercent = mywidth/float(img.size[0])
  hsize = int((float(img.size[1])*float(wpercent)))
  img = img.resize((mywidth,hsize), PIL.Image.ANTIALIAS)
  return np.array(img)
  

In [ ]:
def preprocess(image_path):
    img = image.load_img(image_path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [ ]:
def encode(image):
    image = preprocess(image) # preprocess the image
    fea_vec = model_new.predict(image) # Get the encoding vector for the image
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1]) # reshape from (1, 2048) to (2048, )
    return fea_vec

In [ ]:
maxv=77

In [ ]:
vocab = pd.read_csv(r'/content/Face2Text/data/vocab.csv',header=None)
wordtoin={}
intoword={}
ind=0
for i in vocab[0]:
    wordtoin[i]=ind
    intoword[ind]=i
    ind+=1 

In [ ]:
model_enc = Xception(weights='imagenet')

In [ ]:
model_new = Model(model_enc.input, model_enc.layers[-2].output)

In [ ]:
DIR_PATH = '/content/drive/MyDrive/Text2Face/Dataset/stylegan_imgs/'
seed_descriptions = []

for i in range(14001,15001):
    imagev = encode(DIR_PATH + 'seed_' + str(i)+'.png')
    imagev = imagev.reshape((1,2048))
    desc=greedySearch(imagev)
    # saving seeds and descriptions 
    # path = path.replace('.png','') 
    seed_descriptions.append([i,desc])
    print("seed : ", i)

seed :  14999
seed :  15000


In [ ]:
data = pd.DataFrame(seed_descriptions,columns = ['seed','description'])

In [ ]:
data.head()

,seed,description
0,14001,a pale woman with a strong jaw full lips a sma...
1,14002,a young woman with brown hair and small dark e...
2,14003,a woman with bob blonde hair beady green eyes ...
3,14004,pale skin with slightly droopy features
4,14005,ginger hair and a fringe he has a slightly wea...


In [ ]:
data.to_csv('face_descriptions_14k-15k.csv')

In [ ]:
embeddings= []
count=1
for desc in data['description']:
  print(count)
  count+=1
  embeddings.append(bert_model.encode(desc))

In [ ]:
embeddings = np.array(embeddings)

In [ ]:
np.save('/content/drive/MyDrive/Text2Face/Dataset/f2t_embedding/f2t_embeddings_14k-15k',embeddings)

In [ ]:
data.to_csv('/content/drive/MyDrive/Text2Face/Dataset/f2t_embedding/f2t_embeddings_14k-15k.csv')